In [1]:
import sys
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt

import devices.Camera
import daq
import file
import PySpin
import time
%load_ext autoreload
%autoreload 1

In [2]:
%aimport devices.Camera
cam = devices.Camera.Camera('17583372')

Camera ID: Blackfly BFLY-PGE-31S4M,17583372,FV:1.61.3.00


In [3]:
image = cam.take_photo()

In [4]:
print(image.GetBitsPerPixel())
print(image.GetPixelFormat(), PySpin.PixelFormat_Mono12Packed)
print(image.GetWidth(), image.GetHeight())

12
11 11
2048 1536


In [ ]:
PySpin.PixelFormat_Mono12

In [5]:
converted = image.Convert(PySpin.PixelFormat_Mono16)

SpinnakerException: Spinnaker: Parameter is not initialized. Input image is NULL. [-1009]

In [6]:
cam.close()
del cam

In [3]:
image = cam.take_photo()

In [4]:
cam.set_trigger_settings(False)

In [8]:
camera = cam.cam

In [9]:
info = camera.getCameraInfo()
GigEConfig = camera.getGigEConfig()
config = camera.getConfiguration()
stats = camera.getStats()

In [24]:
vars(GigEConfig)
vars(config)
vars(info)

{'applicationIPAddress': 2852037848,
 'applicationPort': 4211,
 'bayerTileFormat': 0,
 'busNumber': 0,
 'ccpStatus': 2,
 'configROM': {'chipIdHi': 0,
  'chipIdLo': 17529184,
  'nodeVendorId': 483739,
  'pszKeyword': b'',
  'unitSWVer': 258,
  'unitSpecId': 0,
  'unitSubSWVer': 32,
  'vendorUniqueInfo_0': 0,
  'vendorUniqueInfo_1': 0,
  'vendorUniqueInfo_2': 0,
  'vendorUniqueInfo_3': 0},
 'defaultGateway': (0, 0, 0, 0),
 'driverName': b'None',
 'driverType': 8,
 'firmwareBuildTime': b'Sat Nov 14 01:09:14 2015',
 'firmwareVersion': b'1.53.3.0',
 'gigEMajorVersion': 1,
 'gigEMinorVersion': 2,
 'iidcVer': 132,
 'interfaceType': 3,
 'ipAddress': (169, 254, 121, 98),
 'isColorCamera': False,
 'macAddress': (0, 176, 157, 11, 121, 96),
 'maximumBusSpeed': 5,
 'modelName': b'Blackfly BFLY-PGE-50S5M',
 'nodeNumber': 0,
 'pcieBusSpeed': -1,
 'sensorInfo': b'Sony IMX264 (2/3" Mono CMOS)',
 'sensorResolution': b'2448x2048',
 'serialNumber': 17529184,
 'subnetMask': (255, 255, 0, 0),
 'userDefinedN

In [29]:
#PROP = pc2.PROPERTY_TYPE.SHARPNESS
#PROP = pc2.PROPERTY_TYPE.BRIGHTNESS
#PROP = pc2.PROPERTY_TYPE.AUTO_EXPOSURE
#PROP = pc2.PROPERTY_TYPE.BRIGHTNESS
PROP = pc2.PROPERTY_TYPE.SHUTTER
prop = cam.cam.getProperty(PROP)
info = cam.cam.getPropertyInfo(PROP)
print('Present:', info.present)
print('Auto supported:', info.autoSupported)
print('Manual supported:', info.manualSupported)
print('Units:', info.units)
print('onOff:', prop.onOff)
print('autoManual:', prop.autoManualMode)
print('Min:', info.absMin)
print('absValue:', prop.absValue)
print('Max:', info.absMax)
print('Min:', info.min)
print('Value:', prop.valueA)
print('Max:', info.max)

Present: True
Auto supported: True
Manual supported: True
Units: b'Milliseconds'
onOff: True
autoManual: False
Min: 0.059545040130615234
absValue: 0.059545040130615234
Max: 124.02534484863281
Min: 1
Value: 1
Max: 2065


In [4]:
cam.close()
del cam

In [2]:
system = PySpin.System.GetInstance()
cam_list = system.GetCameras()
num_cams = cam_list.GetSize()
print('Number of cameras detected: %i' % num_cams)
for i, cam in enumerate(cam_list):
    nodemap_tl = cam.GetTLDeviceNodeMap()
    node = PySpin.CStringPtr(nodemap_tl.GetNode('DeviceSerialNumber'))
    if PySpin.IsAvailable(node) and PySpin.IsReadable(node):
        value = node.ToString()
    print(value)
del cam
cam_list.Clear()
system.ReleaseInstance()
del system

Number of cameras detected: 1
17529184


In [ ]:
info = cam.cam.getGigEImageSettings()
info.pixelFormat

In [ ]:
pc2.PIXEL_FORMAT.RAW16

In [ ]:
image = cam.take_photo()

In [ ]:
print(len(image.getData())/(2048*1536))
print(image.getData()[0:100])

In [ ]:
%timeit image.save(b'/media/robert/Data_Storage/daq/test.tiff', pc2.IMAGE_FILE_FORMAT.TIFF)

In [ ]:
tiff = plt.imread('/media/robert/Data_Storage/daq/test.tiff')
plt.imshow(tiff/16)
plt.colorbar()
plt.show()

In [ ]:
from PIL import Image
im = Image.open('/media/robert/Data_Storage/daq/test.tiff')
im.info

In [ ]:
shutPROP = pc2.PROPERTY_TYPE.SHUTTER
shut = cam.cam.getProperty(shutPROP)
print(shut)

In [ ]:
cam.cam.disconnect()

In [ ]:
daq.main([['Camera'], [17571186], 'cam_test', 10])

In [ ]:
from PIL import Image
dataSet = 1806150049

image = file.load_IMAGE(17583372, dataSet, 10)
print(file.decode_image_meta(image))

In [ ]:
tiff = plt.imread('/media/robert/Data_Storage/daq/IMAGE/year_2018/month_06/day_15/1806150049/17583372_1806150049_0010.tiff')
plt.imshow(tiff)
plt.colorbar()
plt.show()

In [ ]:
import time
def callback(image, i):
    image.save(bytes('test.tiff', 'utf-8'), pc2.IMAGE_FILE_FORMAT.TIFF)
    print(time.clock())
#cam.cam.startCapture(callback)
cam.start_capture(callback)#, args=(1,))
time.sleep(1)
cam.cam.stopCapture()

In [1]:
# Cython implementation
import sys
sys.path.append('../')
import numpy as np
from calc.camera import convert_image
raw = [i % 200 for i in range(8000000)]
height = 2000
width = 2000
%timeit data = convert_image(raw, height, width)

169 ms ± 209 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [2]:
raw = [i % 200 for i in range(8000000)]
height = 2000
width = 2000
b = bytes(raw)
numpy_array = np.frombuffer(b, dtype=np.uint16)
%timeit bytes(raw)
%timeit np.frombuffer(b, dtype=np.uint16)
%timeit numpy_array.reshape(height, width)
%timeit np.frombuffer(bytes(raw), dtype=np.uint16).reshape(height, width) 

37 ms ± 8.64 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
576 ns ± 5.77 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
234 ns ± 0.845 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
37 ms ± 18.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [1]:
from scipy.ndimage import center_of_mass